In [ ]:
import pytest
import lqrt
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
from dabest._stats_tools import effsize
from dabest import Dabest, TwoGroupsEffectSize, PermutationTest

In [ ]:
from data.mocked_data_test_01 import wellbeing, paired_wellbeing, smoke, likert_control, likert_treatment, a_scores, b_scores, dabest_default_kwargs

test_mean_diff_unpaired

In [ ]:
mean_diff = effsize.func_difference(wellbeing.control, wellbeing.expt,
                                    np.mean, is_paired=False)
assert mean_diff == pytest.approx(5.4)

test_median_diff_unpaired

In [ ]:
median_diff = effsize.func_difference(wellbeing.control, wellbeing.expt,
                                    np.median, is_paired=False)
assert median_diff == pytest.approx(3.5)

test_mean_diff_paired

In [ ]:
mean_diff = effsize.func_difference(paired_wellbeing.pre,
                                    paired_wellbeing.post,
                                    np.mean, is_paired="baseline")
assert mean_diff == pytest.approx(4.10)

test_median_diff_paired

In [ ]:
median_diff = effsize.func_difference(paired_wellbeing.pre,
                                      paired_wellbeing.post,
                                      np.median, is_paired="baseline")
assert median_diff == pytest.approx(4.5)

test_cohens_d_unpaired

In [ ]:
cohens_d = effsize.cohens_d(np.array(wellbeing.control), np.array(wellbeing.expt),
                            is_paired=False)
assert np.round(cohens_d, 2) == pytest.approx(0.47)

test_hedges_g_unpaired

In [ ]:
hedges_g = effsize.hedges_g(np.array(wellbeing.control), np.array(wellbeing.expt),
                                is_paired=False)
assert np.round(hedges_g, 2) == pytest.approx(0.45)

test_cohens_d_paired

In [ ]:
cohens_d = effsize.cohens_d(np.array(paired_wellbeing.pre), np.array(paired_wellbeing.post),
                                is_paired="baseline")
assert np.round(cohens_d, 2) == pytest.approx(0.34)


test_hedges_g_paired

In [ ]:
hedges_g = effsize.hedges_g(paired_wellbeing.pre, paired_wellbeing.post,
                            is_paired="baseline")
assert np.round(hedges_g, 2) == pytest.approx(0.33)

test_cohens_h

In [ ]:
cohens_h = effsize.cohens_h(smoke.low, smoke.high)
assert np.round(cohens_h, 2) == pytest.approx(0.17)

test_cliffs_delta

In [ ]:
likert_delta = effsize.cliffs_delta(likert_treatment, likert_control)
assert likert_delta == pytest.approx(-0.25)

scores_delta = effsize.cliffs_delta(b_scores, a_scores)
assert scores_delta == pytest.approx(0.65)

test_unpaired_stats

In [ ]:
c = wellbeing.control
t = wellbeing.expt

unpaired_es = TwoGroupsEffectSize(c, t, "mean_diff", is_paired=False, proportional=False)

p1 = sp.stats.mannwhitneyu(c, t, alternative="two-sided").pvalue
assert unpaired_es.pvalue_mann_whitney == pytest.approx(p1)

p2 = sp.stats.ttest_ind(c, t, nan_policy='omit').pvalue
assert unpaired_es.pvalue_students_t == pytest.approx(p2)

p3 = sp.stats.ttest_ind(c, t, equal_var=False, nan_policy='omit').pvalue
assert unpaired_es.pvalue_welch == pytest.approx(p3)

test_paired_stats

In [ ]:
before = paired_wellbeing.pre
after = paired_wellbeing.post

paired_es = TwoGroupsEffectSize(before, after, "mean_diff", is_paired="baseline", proportional=False)

p1 = sp.stats.ttest_rel(before, after, nan_policy='omit').pvalue
assert paired_es.pvalue_paired_students_t == pytest.approx(p1)

p2 = sp.stats.wilcoxon(before, after).pvalue
assert paired_es.pvalue_wilcoxon == pytest.approx(p2)

test_median_diff_stats

In [ ]:
c = wellbeing.control
t = wellbeing.expt

es = TwoGroupsEffectSize(c, t, "median_diff", is_paired=False, proportional=False)

p1 = sp.stats.kruskal(c, t, nan_policy='omit').pvalue
assert es.pvalue_kruskal == pytest.approx(p1)

test_ordinal_dominance

In [ ]:
es = TwoGroupsEffectSize(likert_control, likert_treatment, 
                             "cliffs_delta", is_paired=False, proportional=False)
                             
p1 = sp.stats.brunnermunzel(likert_control, likert_treatment).pvalue
assert es.pvalue_brunner_munzel == pytest.approx(p1)

test_unpaired_permutation_test

In [ ]:
perm_test = PermutationTest(wellbeing.control, wellbeing.expt, 
                                effect_size="mean_diff", 
                                is_paired=False)
assert perm_test.pvalue == pytest.approx(0.2976)    

test_paired_permutation_test

In [ ]:
perm_test = PermutationTest(paired_wellbeing.pre, 
                                paired_wellbeing.post, 
                                effect_size="mean_diff", 
                                is_paired="baseline")
assert perm_test.pvalue == pytest.approx(0.0124)

test_lqrt_unpaired

In [ ]:
unpaired_dabest = Dabest(wellbeing, idx=("control", "expt"), 
                             paired=None, id_col=None, 
                             **dabest_default_kwargs)
lqrt_result = unpaired_dabest.mean_diff.lqrt

p1 = lqrt.lqrtest_ind(wellbeing.control, wellbeing.expt,
                      equal_var=True,
                      random_state=12345)

p2 = lqrt.lqrtest_ind(wellbeing.control, wellbeing.expt,
                      equal_var=False,
                      random_state=12345)

assert lqrt_result.pvalue_lqrt_equal_var[0] == pytest.approx(p1.pvalue)
assert lqrt_result.pvalue_lqrt_unequal_var[0] == pytest.approx(p2.pvalue)

test_lqrt_paired

In [ ]:
paired_dabest = Dabest(paired_wellbeing, idx=("pre", "post"),
                           paired="baseline", id_col="ID",
                           **dabest_default_kwargs)
lqrt_result = paired_dabest.mean_diff.lqrt

p1 = lqrt.lqrtest_rel(paired_wellbeing.pre, paired_wellbeing.post, 
             random_state=12345)

assert lqrt_result.pvalue_paired_lqrt[0] == pytest.approx(p1.pvalue)